In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

%matplotlib inline

%cd ../..

!hostname

/p/fastdata/pli/Private/oberstrass1/datasets/vervet1818-3d
jrlogin11.jureca


In [2]:
import h5py as h5
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
def load_volume(clustering, method):
    path = f"data/aa/volume/{clustering}/{method}/{clustering}.h5"
    with h5.File(path, 'r') as f:
        volume = f['volume'][:]
    return path, volume

def iou_cluster(volume, cluster_ix, ignore_ix=0):
    vol_rolled = np.roll(volume, 1, axis=1)
    vol_clip = volume[:, 1:, :]
    vol_rolled_clip = vol_rolled[:, 1:, :]

    hits1 = vol_rolled_clip[vol_clip != ignore_ix] == cluster_ix
    hits2 = vol_clip[vol_clip != ignore_ix] == cluster_ix

    intersection = np.sum(hits1 & hits2)
    union = np.sum(hits1 | hits2)

    return intersection / union

In [17]:
methods = [
    "cl-2d_all_augmentations_circle_small",
    "cl-2d_only_flip_circle_small",
    "cl-3d_no_augmentations_sphere_small",
    "cl-3d_only_scale-attenuation_sphere_small",
    "cl-2d_no_augmentations_circle_small",
    "cl-2d_only_scale-attenuation_circle_small",
    "cl-3d_only_affine_sphere_small",
    "cl-3d_only_scale-thickness_sphere_small",
    "cl-2d_only_affine_circle_small",
    "cl-2d_only_scale-thickness_circle_small",
    "cl-3d_only_blur_sphere_small",
    "cl-2d_only_blur_circle_small",
    "cl-3d_all_augmentations_sphere_small",
    "cl-3d_only_flip_sphere_small"
]

clusterings = [
    "kmeans_pca80_s1_2",
    "kmeans_pca80_s1_8",
    "kmeans_pca80_s1_32",
    "kmeans_pca80_s1_128"
]

models = ['cl-3d', 'cl-2d']

###

from tqdm import tqdm

iou_frame = pd.DataFrame()

for m in tqdm(methods):
    for c in clusterings:
        print(m, c)

        path, volume = load_volume(c, m)
        print(path)

        cluster_ix = np.unique(volume)[1:]

        ious = []
        for c_ix in cluster_ix:
            ious.append(iou_cluster(volume, c_ix))

        mean_iou = np.mean(ious)

        print("Mean IoU:", mean_iou)

        iou_frame = iou_frame.append({'method': m, 'clustering': c, 'iou_list': ious, 'mean_iou': mean_iou}, ignore_index=True)

iou_frame = iou_frame.sort_values(['clustering', 'method']).reset_index(drop=True)

def method_split(method):
    model = None
    for m in models:
        if method.startswith(m):
            model = m
    context = method.split(model)[-1]
    if len(context) > 0 and context[0] == "_":
        context = context[1:]
    return model, context

for k, r in iou_frame.iterrows():
    m, c = method_split(r['method'])
    iou_frame.loc[k, 'method'] = m
    iou_frame.loc[k, 'context'] = c

  0%|          | 0/14 [00:00<?, ?it/s]

cl-2d_all_augmentations_circle_small kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/cl-2d_all_augmentations_circle_small/kmeans_pca80_s1_2.h5
Mean IoU: 0.9516172367039993
cl-2d_all_augmentations_circle_small kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/cl-2d_all_augmentations_circle_small/kmeans_pca80_s1_8.h5
Mean IoU: 0.6118931041783807
cl-2d_all_augmentations_circle_small kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/cl-2d_all_augmentations_circle_small/kmeans_pca80_s1_32.h5
Mean IoU: 0.363044108403496
cl-2d_all_augmentations_circle_small kmeans_pca80_s1_128
data/aa/volume/kmeans_pca80_s1_128/cl-2d_all_augmentations_circle_small/kmeans_pca80_s1_128.h5


  7%|▋         | 1/14 [00:13<02:52, 13.26s/it]

Mean IoU: 0.19959032352491562
cl-2d_only_flip_circle_small kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/cl-2d_only_flip_circle_small/kmeans_pca80_s1_2.h5
Mean IoU: 0.8677020850750181
cl-2d_only_flip_circle_small kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/cl-2d_only_flip_circle_small/kmeans_pca80_s1_8.h5
Mean IoU: 0.5296088041958134
cl-2d_only_flip_circle_small kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/cl-2d_only_flip_circle_small/kmeans_pca80_s1_32.h5
Mean IoU: 0.2980119427972931
cl-2d_only_flip_circle_small kmeans_pca80_s1_128
data/aa/volume/kmeans_pca80_s1_128/cl-2d_only_flip_circle_small/kmeans_pca80_s1_128.h5


 14%|█▍        | 2/14 [00:26<02:38, 13.23s/it]

...

Mean IoU: 0.24900135717317073
cl-3d_all_augmentations_sphere_small kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/cl-3d_all_augmentations_sphere_small/kmeans_pca80_s1_2.h5
Mean IoU: 0.9589534537075929
cl-3d_all_augmentations_sphere_small kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/cl-3d_all_augmentations_sphere_small/kmeans_pca80_s1_8.h5
Mean IoU: 0.7320998978672544
cl-3d_all_augmentations_sphere_small kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/cl-3d_all_augmentations_sphere_small/kmeans_pca80_s1_32.h5
Mean IoU: 0.5078094392047703
cl-3d_all_augmentations_sphere_small kmeans_pca80_s1_128
data/aa/volume/kmeans_pca80_s1_128/cl-3d_all_augmentations_sphere_small/kmeans_pca80_s1_128.h5


 93%|█████████▎| 13/14 [02:51<00:13, 13.22s/it]

Mean IoU: 0.3150065895640148
cl-3d_only_flip_sphere_small kmeans_pca80_s1_2
data/aa/volume/kmeans_pca80_s1_2/cl-3d_only_flip_sphere_small/kmeans_pca80_s1_2.h5
Mean IoU: 0.9544824213989661
cl-3d_only_flip_sphere_small kmeans_pca80_s1_8
data/aa/volume/kmeans_pca80_s1_8/cl-3d_only_flip_sphere_small/kmeans_pca80_s1_8.h5
Mean IoU: 0.7118299013631633
cl-3d_only_flip_sphere_small kmeans_pca80_s1_32
data/aa/volume/kmeans_pca80_s1_32/cl-3d_only_flip_sphere_small/kmeans_pca80_s1_32.h5
Mean IoU: 0.4887630640942432
cl-3d_only_flip_sphere_small kmeans_pca80_s1_128
data/aa/volume/kmeans_pca80_s1_128/cl-3d_only_flip_sphere_small/kmeans_pca80_s1_128.h5


100%|██████████| 14/14 [03:05<00:00, 13.22s/it]

Mean IoU: 0.3050274182106454


In [18]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_2'].sort_values(['method', 'mean_iou'])

,method,clustering,iou_list,mean_iou,context
18,cl-2d,kmeans_pca80_s1_2,"[0.8559824800411234, 0.8794216901089127]",0.867702,only_flip_circle_small
19,cl-2d,kmeans_pca80_s1_2,"[0.881768885119381, 0.8909286880960965]",0.886349,only_scale-attenuation_circle_small
16,cl-2d,kmeans_pca80_s1_2,"[0.9130279551327981, 0.8799802944199512]",0.896504,only_affine_circle_small
15,cl-2d,kmeans_pca80_s1_2,"[0.8991921741681072, 0.9072186024103577]",0.903205,no_augmentations_circle_small
17,cl-2d,kmeans_pca80_s1_2,"[0.9026492762174629, 0.9097963772352032]",0.906223,only_blur_circle_small
20,cl-2d,kmeans_pca80_s1_2,"[0.9352849829735592, 0.9587591222030346]",0.947022,only_scale-thickness_circle_small
14,cl-2d,kmeans_pca80_s1_2,"[0.9389567420730969, 0.9642777313349018]",0.951617,all_augmentations_circle_small
22,cl-3d,kmeans_pca80_s1_2,"[0.6542999925482902, 0.9048575329422716]",0.779579,no_augmentations_sphere_small
27,cl-3d,kmeans_pca80_s1_2,"[0.9166997533639076, 0.8555811797352466]",0.886140,only_scale-thickness_sphere_small
24,cl-3d,kmeans_pca80_s1_2,"[0.85834002559646, 0.9160684095552127]",0.887204,only_blur_sphere_small


In [19]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_8'].sort_values(['method', 'mean_iou'])

,method,clustering,iou_list,mean_iou,context
43,cl-2d,kmeans_pca80_s1_8,"[0.5196583303295662, 0.4721394116670525, 0.427...",0.492756,no_augmentations_circle_small
46,cl-2d,kmeans_pca80_s1_8,"[0.731124147393094, 0.39059226117466483, 0.626...",0.529609,only_flip_circle_small
45,cl-2d,kmeans_pca80_s1_8,"[0.5826922151374261, 0.43245743482864685, 0.59...",0.560038,only_blur_circle_small
44,cl-2d,kmeans_pca80_s1_8,"[0.6974149724878457, 0.630041588283204, 0.4404...",0.567898,only_affine_circle_small
42,cl-2d,kmeans_pca80_s1_8,"[0.4494295047978885, 0.7041129863485581, 0.801...",0.611893,all_augmentations_circle_small
48,cl-2d,kmeans_pca80_s1_8,"[0.7902975695086305, 0.6403646697654704, 0.617...",0.620978,only_scale-thickness_circle_small
47,cl-2d,kmeans_pca80_s1_8,"[0.8020837348546171, 0.5905561477357258, 0.636...",0.645263,only_scale-attenuation_circle_small
50,cl-3d,kmeans_pca80_s1_8,"[0.7471633942296053, 0.5346329018544, 0.768633...",0.664305,no_augmentations_sphere_small
52,cl-3d,kmeans_pca80_s1_8,"[0.7073568823114208, 0.6333096761918232, 0.838...",0.673840,only_blur_sphere_small
51,cl-3d,kmeans_pca80_s1_8,"[0.37385365661712805, 0.7434835497713469, 0.59...",0.696365,only_affine_sphere_small


In [20]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_32'].sort_values(['method', 'mean_iou'])

,method,clustering,iou_list,mean_iou,context
32,cl-2d,kmeans_pca80_s1_32,"[0.156899738179578, 0.3760730613632128, 0.4243...",0.298012,only_flip_circle_small
30,cl-2d,kmeans_pca80_s1_32,"[0.23239282925591162, 0.2022276318458696, 0.26...",0.312335,only_affine_circle_small
29,cl-2d,kmeans_pca80_s1_32,"[0.2906454953400069, 0.32888070317889406, 0.15...",0.320149,no_augmentations_circle_small
28,cl-2d,kmeans_pca80_s1_32,"[0.22710070076934616, 0.46872250401117954, 0.3...",0.363044,all_augmentations_circle_small
31,cl-2d,kmeans_pca80_s1_32,"[0.5542028022373595, 0.4125078911748409, 0.427...",0.384108,only_blur_circle_small
34,cl-2d,kmeans_pca80_s1_32,"[0.49482905233036834, 0.39998622699364267, 0.2...",0.421121,only_scale-thickness_circle_small
33,cl-2d,kmeans_pca80_s1_32,"[0.2705385419504683, 0.41601155321039074, 0.66...",0.425518,only_scale-attenuation_circle_small
38,cl-3d,kmeans_pca80_s1_32,"[0.3993315274936959, 0.45003974720775863, 0.82...",0.472165,only_blur_sphere_small
36,cl-3d,kmeans_pca80_s1_32,"[0.4203474634028886, 0.5707461468663871, 0.323...",0.472777,no_augmentations_sphere_small
41,cl-3d,kmeans_pca80_s1_32,"[0.3264248579198055, 0.421297636910873, 0.6245...",0.483999,only_scale-thickness_sphere_small


In [21]:
iou_frame.loc[iou_frame['clustering'] == 'kmeans_pca80_s1_128'].sort_values(['method', 'mean_iou'])

,method,clustering,iou_list,mean_iou,context
2,cl-2d,kmeans_pca80_s1_128,"[0.1507470040039137, 0.18456089690791577, 0.34...",0.165230,only_affine_circle_small
4,cl-2d,kmeans_pca80_s1_128,"[0.11628013777267508, 0.2707090947770054, 0.10...",0.172647,only_flip_circle_small
0,cl-2d,kmeans_pca80_s1_128,"[0.11853379281773158, 0.27317741554453107, 0.0...",0.199590,all_augmentations_circle_small
1,cl-2d,kmeans_pca80_s1_128,"[0.13262654806835208, 0.14674621351115386, 0.2...",0.206688,no_augmentations_circle_small
3,cl-2d,kmeans_pca80_s1_128,"[0.2044578758445946, 0.2130254058560989, 0.299...",0.249001,only_blur_circle_small
5,cl-2d,kmeans_pca80_s1_128,"[0.10395474899835022, 0.3558004694470784, 0.14...",0.273036,only_scale-attenuation_circle_small
6,cl-2d,kmeans_pca80_s1_128,"[0.1963004144344486, 0.1890865480711573, 0.370...",0.277886,only_scale-thickness_circle_small
11,cl-3d,kmeans_pca80_s1_128,"[0.32350222079305424, 0.13202487803061339, 0.3...",0.305027,only_flip_sphere_small
9,cl-3d,kmeans_pca80_s1_128,"[0.2773476131895699, 0.17614196803449692, 0.43...",0.305961,only_affine_sphere_small
8,cl-3d,kmeans_pca80_s1_128,"[0.32087344419918873, 0.1670135006035651, 0.22...",0.312559,no_augmentations_sphere_small
